In [2]:
import os
import numpy as np
import mne
from mne.preprocessing import ICA
from mne_icalabel import label_components
from pyriemann.utils.mean import mean_riemann
from pyriemann.preprocessing import Whitening


In [16]:
TARGET_SFREQ = 250          # Common sampling rate
L_FREQ = 0.5                # Band-pass low
H_FREQ = 50.0               # Band-pass high
N_COMPONENTS = 0.99         # ICA variance
RANDOM_STATE = 42


In [17]:
def load_rest_bids_subject(subject_path):
    """
    Loads EEGLAB .set file from BIDS-style rest dataset
    """
    eeg_dir = os.path.join(subject_path, "eeg")
    set_file = [f for f in os.listdir(eeg_dir) if f.endswith(".set")][0]
    raw = mne.io.read_raw_eeglab(os.path.join(eeg_dir, set_file), preload=True)
    return raw


In [18]:
def load_interval_subject(vhdr_path):
    """
    Loads BrainVision EEG (.vhdr)
    """
    raw = mne.io.read_raw_brainvision(vhdr_path, preload=True)
    return raw


In [27]:
def drop_channels_without_positions(raw):
    """
    ICLabel REQUIRES all channels to have valid 3D locations.
    This function FORCE-DROPS any channel without a position.
    """

    ch_pos = raw.get_montage().get_positions()["ch_pos"]

    bad_chs = []
    for ch in raw.ch_names:
        if ch not in ch_pos or np.any(np.isnan(ch_pos[ch])):
            bad_chs.append(ch)

    if len(bad_chs) > 0:
        print("Dropping channels without positions:", bad_chs)
        raw.drop_channels(bad_chs)

    return raw


In [28]:
def standardized_preprocessing(raw):

    # ---- Keep EEG channels only ----
    raw.pick_types(
        eeg=True,
        eog=False,
        ecg=False,
        emg=False,
        stim=False,
        misc=False
    )

    # ---- Set montage FIRST ----
    raw.set_montage("standard_1020", match_case=False, on_missing="ignore")

    # ---- FORCE drop channels without 3D positions ----
    raw = drop_channels_without_positions(raw)

    # ---- Resample ----
    raw.resample(250)

    # ---- Band-pass filter ----
    raw.filter(0.5, 50.0, fir_design="firwin")

    # ---- Common Average Reference ----
    raw.set_eeg_reference("average", projection=False)

    return raw


In [29]:
def artifact_correction(raw):

    ica = ICA(
        n_components=0.99,
        random_state=42,
        max_iter="auto"
    )
    ica.fit(raw)

    labels = label_components(raw, ica, method="iclabel")
    ic_labels = labels["labels"]

    exclude = [
        i for i, lab in enumerate(ic_labels)
        if lab in ["eye blink", "muscle artifact", "heart beat"]
    ]

    ica.exclude = exclude
    raw = ica.apply(raw)

    return raw


In [21]:
def subject_zscore(raw):
    """
    Z-score normalization per subject
    """
    data = raw.get_data()
    mean = data.mean(axis=1, keepdims=True)
    std = data.std(axis=1, keepdims=True)
    data_z = (data - mean) / std
    raw._data = data_z
    return raw


In [22]:
def riemannian_recentering(epochs):
    """
    Re-center data on Riemannian manifold
    """
    covs = mne.compute_covariance(epochs, method="oas")
    cov_matrices = covs.data

    mean_cov = mean_riemann(cov_matrices)

    whitener = Whitening(metric="riemann")
    covs_whitened = whitener.fit_transform(cov_matrices)

    return covs_whitened


In [30]:
def prepare_for_ica(raw):
    """
    Keep only true EEG scalp channels for ICA/ICLabel
    """
    # Pick EEG channels only
    raw.pick_types(
        eeg=True,
        eog=False,
        ecg=False,
        emg=False,
        misc=False,
        stim=False
    )

    # Drop known non-scalp channels if still present
    bad_names = ["I1", "I2", "HEOG", "VEOG", "EOG"]
    drop_chs = [ch for ch in raw.ch_names if ch.upper() in bad_names]
    if len(drop_chs) > 0:
        raw.drop_channels(drop_chs)

    return raw


In [31]:
raw.set_montage("standard_1020", on_missing="ignore")


<RawEEGLAB | sub-009_task-Rest_eeg.fdt, 63 x 62040 (248.2 s), ~29.9 MiB, data loaded>

In [32]:
raw = prepare_for_ica(raw)
raw.set_montage("standard_1020", match_case=False)


NOTE: pick_types() is a legacy function. New code should use inst.pick(...).


<RawEEGLAB | sub-009_task-Rest_eeg.fdt, 61 x 62040 (248.2 s), ~28.9 MiB, data loaded>

def artifact_correction(raw):
    """
    ICA + ICLabel + bad channel interpolation (ROBUST)
    """

    # ICA
    ica = ICA(
        n_components=0.99,
        random_state=42,
        max_iter="auto"
    )
    ica.fit(raw)

    # ICLabel
    labels = label_components(raw, ica, method="iclabel")
    ic_labels = labels["labels"]

    # Reject non-brain ICs
    exclude_idx = [
        idx for idx, lab in enumerate(ic_labels)
        if lab in ["eye blink", "muscle artifact", "heart beat"]
    ]

    ica.exclude = exclude_idx
    raw = ica.apply(raw)

    # ---- Bad channel detection ----
    data = raw.get_data()
    ch_var = np.var(data, axis=1)
    threshold = np.percentile(ch_var, 99)

    raw.info["bads"] = [
        raw.ch_names[i]
        for i in range(len(ch_var))
        if ch_var[i] > threshold
    ]

    # ---- Interpolate bad channels ----
    if len(raw.info["bads"]) > 0:
        raw.interpolate_bads(reset_bads=True)

    return raw


In [33]:
rest_raws = []

for subj in sorted(os.listdir(rest_root)):
    if subj.startswith("sub-"):
        raw = load_rest_bids_subject(os.path.join(rest_root, subj))
        raw = standardized_preprocessing(raw)

        print("Before ICA:", raw.ch_names)  # DEBUG LINE

        raw = artifact_correction(raw)
        raw = subject_zscore(raw)
        rest_raws.append(raw)


Reading c:\Users\Usha Sri\OneDrive\Documents\Parkinson_Project\ds004584-download\sub-001\eeg\sub-001_task-Rest_eeg.fdt
Reading 0 ... 140829  =      0.000 ...   281.658 secs...
NOTE: pick_types() is a legacy function. New code should use inst.pick(...).


C:\Users\Usha Sri\AppData\Local\Temp\ipykernel_944\3293849432.py:7: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these events.
  raw = mne.io.read_raw_eeglab(os.path.join(eeg_dir, set_file), preload=True)


Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.5 - 50 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.50
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 0.25 Hz)
- Upper passband edge: 50.00 Hz
- Upper transition bandwidth: 12.50 Hz (-6 dB cutoff frequency: 56.25 Hz)
- Filter length: 1651 samples (6.604 s)

EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.
Before ICA: ['Fp1', 'Fz', 'F3', 'F7', 'FT9', 'FC5', 'FC1', 'C3', 'T7', 'TP9', 'CP5', 'CP1', 'P3', 'P7', 'O1', 'Oz', 'O2', 'P4', 'P8', 'TP10', 'CP6', 'CP2', 'Cz', 'C4', 'T8', 'FT10', 'FC6', 'FC2', 'F4', 'F8', 'Fp2', 'AF7', 'AF3', 'AFz', 'F1', 'F5', 'FT7', 'FC3', 'C1', 'C5', 'TP7', 'CP3', 'P1', 'P5', 'PO7', 'PO3', 'POz', 'PO4

C:\Users\Usha Sri\AppData\Local\Temp\ipykernel_944\4144841098.py:10: RuntimeWarning: The provided Raw instance is not filtered between 1 and 100 Hz. ICLabel was designed to classify features extracted from an EEG dataset bandpass filtered between 1 and 100 Hz (see the 'filter()' method for Raw and Epochs instances).
  labels = label_components(raw, ica, method="iclabel")
C:\Users\Usha Sri\AppData\Local\Temp\ipykernel_944\4144841098.py:10: RuntimeWarning: The provided ICA instance was fitted with a 'fastica' algorithm. ICLabel was designed with extended infomax ICA decompositions. To use the extended infomax algorithm, use the 'mne.preprocessing.ICA' instance with the arguments 'ICA(method='infomax', fit_params=dict(extended=True))' (scikit-learn) or 'ICA(method='picard', fit_params=dict(ortho=False, extended=True))' (python-picard).
  labels = label_components(raw, ica, method="iclabel")


Applying ICA to Raw instance
    Transforming to ICA space (38 components)
    Zeroing out 24 ICA components
    Projecting back using 63 PCA components
Reading c:\Users\Usha Sri\OneDrive\Documents\Parkinson_Project\ds004584-download\sub-002\eeg\sub-002_task-Rest_eeg.fdt
Reading 0 ... 163019  =      0.000 ...   326.038 secs...
NOTE: pick_types() is a legacy function. New code should use inst.pick(...).


...

C:\Users\Usha Sri\AppData\Local\Temp\ipykernel_944\4144841098.py:10: RuntimeWarning: The provided Raw instance is not filtered between 1 and 100 Hz. ICLabel was designed to classify features extracted from an EEG dataset bandpass filtered between 1 and 100 Hz (see the 'filter()' method for Raw and Epochs instances).
  labels = label_components(raw, ica, method="iclabel")
C:\Users\Usha Sri\AppData\Local\Temp\ipykernel_944\4144841098.py:10: RuntimeWarning: The provided ICA instance was fitted with a 'fastica' algorithm. ICLabel was designed with extended infomax ICA decompositions. To use the extended infomax algorithm, use the 'mne.preprocessing.ICA' instance with the arguments 'ICA(method='infomax', fit_params=dict(extended=True))' (scikit-learn) or 'ICA(method='picard', fit_params=dict(ortho=False, extended=True))' (python-picard).
  labels = label_components(raw, ica, method="iclabel")


Applying ICA to Raw instance
    Transforming to ICA space (26 components)
    Zeroing out 5 ICA components
    Projecting back using 61 PCA components
Reading c:\Users\Usha Sri\OneDrive\Documents\Parkinson_Project\ds004584-download\sub-040\eeg\sub-040_task-Rest_eeg.fdt
Reading 0 ... 68659  =      0.000 ...   137.318 secs...
NOTE: pick_types() is a legacy function. New code should use inst.pick(...).


C:\Users\Usha Sri\AppData\Local\Temp\ipykernel_944\3293849432.py:7: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these events.
  raw = mne.io.read_raw_eeglab(os.path.join(eeg_dir, set_file), preload=True)


Dropping channels without positions: ['I1', 'I2']
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.5 - 50 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.50
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 0.25 Hz)
- Upper passband edge: 50.00 Hz
- Upper transition bandwidth: 12.50 Hz (-6 dB cutoff frequency: 56.25 Hz)
- Filter length: 1651 samples (6.604 s)

EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.
Before ICA: ['Fp1', 'Fz', 'F3', 'F7', 'Iz', 'FC5', 'FC1', 'C3', 'T7', 'TP9', 'CP5', 'CP1', 'P3', 'P7', 'O1', 'Oz', 'O2', 'P4', 'P8', 'TP10', 'CP6', 'CP2', 'Cz', 'C4', 'T8', 'FT10', 'FC6', 'FC2', 'F4', 'F8', 'Fp2', 'AF7', 'AF3', 'AFz', 'F1', 'F5', 'FT7', 'FC3', 'C1', 'C5', 'T

RuntimeError: One PCA component captures most of the explained variance (99.60226260952169%), your threshold results in 1 component. You should select a higher value.

In [ ]:
interval_root = "./JamesCavanagh/PD_Dataset_timing"

interval_raws = []

for file in os.listdir(interval_root):
    if file.endswith(".vhdr"):
        raw = load_interval_subject(os.path.join(interval_root, file))
        raw = standardized_preprocessing(raw)
        raw = artifact_correction(raw)
        raw = subject_zscore(raw)
        interval_raws.append(raw)


In [ ]:
def epoch_data(raw):
    epochs = mne.make_fixed_length_epochs(
        raw,
        duration=2.0,
        overlap=1.0,
        preload=True
    )
    return epochs

rest_epochs = [epoch_data(r) for r in rest_raws]
interval_epochs = [epoch_data(r) for r in interval_raws]


In [ ]:
rest_covs = [riemannian_recentering(ep) for ep in rest_epochs]
interval_covs = [riemannian_recentering(ep) for ep in interval_epochs]
